In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [12]:
!pip install pymorphy2

In [13]:
import numpy as np
import pandas as pd
import re
from wordcloud import WordCloud

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_ru = stopwords.words("russian")
from nltk.stem import WordNetLemmatizer
import pymorphy2
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
#import transformers
#import torch
#import tensorflow as tf
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
#from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
#from transformers import BertTokenizer, BertModel, BertConfig

In [4]:
train = pd.read_csv('../input/scan-classification-challange/df_train.csv')
train.sample(3)

In [5]:
train.info()

In [7]:
# Посмотрим, что из себя представляет значение с текстом

train['text'][12]

In [8]:
# Удалим дубликаты

train.drop_duplicates(subset={'text'}, inplace=True) 
train.shape

In [9]:
# Посмотрим количество текста по каждому классу

train['class'].value_counts()

In [16]:
# Видно, что данные сильно не сбалансированны

from collections import Counter
counter = Counter(train['class'])
plt.figure(figsize=(15, 6))
plt.bar(counter.keys(), counter.values(), width=2)
plt.xticks(rotation=90)
plt.show()

In [18]:
# # Названия категорий переводим в числовой формат и записываем в отдельный столбец

train['encoded_cat'] = train['class'].astype('category').cat.codes
train.sample(5)

In [19]:
train[['class', 'encoded_cat']].value_counts()

In [17]:
test = pd.read_csv('../input/scan-classification-challange/df_test.csv', index_col=0)
test.sample(3)

In [23]:
#Средняя длина, стандартное отклонение, макс.длина столбца text
lens = train.text.str.len()
lens.mean(), lens.std(), lens.max()

In [68]:
pd.unique(train['class'])

In [67]:
# Смотрим кол-во по каждому классу
train['class'].value_counts()

In [30]:
# Посмотрим на состав имеющихся стоп-слов
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_ru = stopwords.words("russian")
print(stopwords.words("russian"))

In [51]:
# Теперь выведем все слова длиной менее 3-х символов и дополним список стоп-слов
# пробовал искать среди стопслов равных 4-м, но там получаем слова 'газа', 'прав', 'дела', 'Рост'
# как мне кажется данные слова, могут быть полезны для score
stopworlds_new = set()
mas_stop = set()
for words in train['text']:
    for i in words.split():
        if len(i) <= 3:
            mas_stop.add(i)
print(mas_stop)

In [52]:
# Создаем новый список стоп-слов

stopworlds_new = set(stopwords_ru).union(mas_stop)

In [ ]:
# Создадим функцию по очистке данных. Будем переводить слова в нижний регистр, 
# удалять стоп слова, удалять числа и раздичные знаки которые не несут смысловой нагрузки. 
# Все слова преобразуем к их первоначалоной форме (Лемматизация)

morph = pymorphy2.MorphAnalyzer()
patterns = "[A-Z|a-z|0-9!#$%&'()*+,./:“″;”<=>?@[\]^_`{|}~—\"\-•–«»]+"
#stops = set(stopwords.words("russian"))
def clean(text):
    text = text.lower()
    text = re.sub(patterns, ' ', text)
    tokens = []
    for token in text.split():
        if token and token not in stopworlds_new:
            token = token.strip()
            token = morph.normal_forms(token)[0]  # Лемматизация
            #token = stemmer.stem(token) # Стеммизация
            tokens.append(token)
    return ' '.join(tokens)

In [75]:
# Применим функцию очистки к train
train['clean'] = train['text'].apply(lambda x: clean(x))
train[['clean', 'text']]

In [72]:
# Выведем список слов которые часто встречаются, при этом исключим стоп слова

words = list( train['clean'].values)

split_words=[]
for word in words:
    lo_w=[]
    list_of_words=str(word).split()
    for w in list_of_words:
        if w not in stopwords_ru:
            lo_w.append(w)
    split_words.append(lo_w)
allwords = []
for wordlist in split_words:
    allwords += wordlist

In [74]:
# Получим 10 часто встречающихся слов

from nltk import FreqDist
word_freq = FreqDist(allwords).most_common(10)
word_freq

In [66]:
# Выведем облако слов

comment_words = ''
comment_words += " ".join(allwords)+" " 
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
               # stopwords = stopwords,
                stopwords = stopworlds_new,
                min_font_size = 10).generate(comment_words)                     
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)